In [1]:
# ML_bot training

from models.ML_bot import train_model

train_model("NN")

Dataset Statistics:
Samples of wins: 376438
Samples of losses: 376438
Training a Complex (Neural Network) model.
Starting training phase...
Iteration 1, loss = 0.61848127
Validation score: 0.673042
Iteration 2, loss = 0.59507201
Validation score: 0.682526
Iteration 3, loss = 0.58532019
Validation score: 0.691624
Iteration 4, loss = 0.57682119
Validation score: 0.698876
Iteration 5, loss = 0.56761689
Validation score: 0.706354
Iteration 6, loss = 0.55762880
Validation score: 0.712358
Iteration 7, loss = 0.54776148
Validation score: 0.719371
Iteration 8, loss = 0.53834420
Validation score: 0.725282
Iteration 9, loss = 0.52949730
Validation score: 0.730236
Iteration 10, loss = 0.52133117
Validation score: 0.735150
Iteration 11, loss = 0.51403773
Validation score: 0.739613
Iteration 12, loss = 0.50754769
Validation score: 0.742482
Iteration 13, loss = 0.50205138
Validation score: 0.745590
Iteration 14, loss = 0.49731089
Validation score: 0.747463
Iteration 15, loss = 0.49329655
Validation 

In [1]:
# Round robin tournament, 3 bots

from models.ML_bot import MLPlayingBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import RandBot
from models.Deepbot import DeepLearningBot
from schnapsen.game import SchnapsenGamePlayEngine
import pathlib
import random


engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

model_path = "./models/model_20250112_044306_epochs25_batch128_lr0.004.pt"

rng = random.Random(42)

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")
bot3 = RdeepBot(num_samples=5, depth=2, rand=rng, name="RdeepBot")
bot4 = DeepLearningBot(model_path=model_path, input_size=173, hidden_size=64, name="DLbot")

def round_robin_tournament():
    wins_MLBot = 0
    wins_RDeep = 0
    wins_Randy = 0
    for i in range (500):

        if i + 1 % 500 == 0:
            print(f"Current progress: game {i}")

        winner_id, game_points, score = engine.play_game(bot1, bot4, random.Random(i))

        if winner_id._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id2, game_points2, score2 = engine.play_game(bot1, bot3, random.Random(i))

        if winner_id2._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id2._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id3, game_points3, score3 = engine.play_game(bot4, bot3, random.Random(i))

        if winner_id3._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id3._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1



    print (f"DLBot: {wins_MLBot}")
    print (f"Randy: {wins_Randy}")
    print (f"Rdeep: {wins_RDeep}")

round_robin_tournament()

/home/roti/workspace/uni_projects/cardbot/models/Deepbot.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=self.device)


Current progress: game 0
DLBot: 373
Randy: 518
Rdeep: 609


In [3]:
# 1v1 arena

import sys
sys.path.append("./models")
from schnapsen.bots import RdeepBot
from schnapsen.bots import RandBot
from schnapsen.game import SchnapsenGamePlayEngine
import random
from mccfr_bot import MCCFRBot


engine = SchnapsenGamePlayEngine()

rng = random.Random(42)

bot2 = RandBot(rng, "RandBot")
bot3 = RdeepBot(num_samples=10, depth=3, rand=rng, name="RdeepBot")
bot5 = RdeepBot(num_samples=20, depth=5, rand=rng, name="RdeepBot2")
bot6 = MCCFRBot(name= "MCCFR")

def matches_1v1(bot1, bot2):
    wins_bot1 = 0
    wins_bot2 = 0
    
    for i in range (10000):

        if i + 1 % 500 == 0:
            print(f"Current progress: game {i}")

        winner_id, game_points, score = engine.play_game(bot1, bot2, random.Random(i))

        if winner_id._Bot__name == str(bot1):
            wins_bot1 +=1
        elif winner_id._Bot__name == str(bot2):
            wins_bot2 +=1


    print (f"{str(bot1)} wins: {wins_bot1}")
    print (f"{str(bot2)} wins: {wins_bot2}")

matches_1v1(bot2, bot6)


RandBot wins: 5761
MCCFR wins: 4239


In [1]:
#Data generation

from data_gen.data_generation import create_replay_memory_dataset
import random
from schnapsen.bots import RandBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import BullyBot

rng = random.Random(42)

bot1 = RandBot(rng)
bot2 = RdeepBot(num_samples=8, depth=4, rand=rng)
bot4 = BullyBot(rng)

create_replay_memory_dataset(bot1=bot2, bot2=bot2, num_of_games = 1, parallel = False, overwrite = True)

In [2]:
from schnapsen.game import SchnapsenGamePlayEngine
from mccfr_bot import MCCFRBot, train_mccfr
import sys

# Create the bots and engine
bot1 = MCCFRBot("MCCFR_Bot1")
bot2 = MCCFRBot("MCCFR_Bot2")
engine = SchnapsenGamePlayEngine()

# Run training
num_iterations = 100000
print(f"Starting training for {num_iterations} iterations...")
train_mccfr(num_iterations, bot1, bot2, engine)

# Save the trained models
bot1.save_model("mccfr_bot1.pkl")
bot2.save_model("mccfr_bot2.pkl")
print("Training complete. Models saved.")

Starting training for 100000 iterations...
Training iteration 0/100000
Training iteration 10/100000
Training iteration 20/100000
Training iteration 30/100000
Training iteration 40/100000
Training iteration 50/100000
Training iteration 60/100000
Training iteration 70/100000
Training iteration 80/100000
Training iteration 90/100000
Training iteration 100/100000
Training iteration 110/100000
Training iteration 120/100000
Training iteration 130/100000
Training iteration 140/100000
Training iteration 150/100000
Training iteration 160/100000
Training iteration 170/100000
Training iteration 180/100000
Training iteration 190/100000
Training iteration 200/100000
Training iteration 210/100000
Training iteration 220/100000
Training iteration 230/100000
Training iteration 240/100000
Training iteration 250/100000
Training iteration 260/100000
Training iteration 270/100000
Training iteration 280/100000
Training iteration 290/100000
Training iteration 300/100000
Training iteration 310/100000
Training